In [4]:
import boto3

In [6]:
s3 = boto3.client('s3')

In [8]:
# ACL='private'|'public-read'|'public-read-write'|'authenticated-read',
#       Bucket='string',
#       CreateBucketConfiguration={
#           'LocationConstraint': 'EU'|'eu-west-1'|'us-west-1'|'us-west-2'|'ap-south-1'|'ap-southeast-1'|'ap-southeast-2'|'ap-northeast-1'|'sa-east-1'|'cn-north-1'|'eu-central-1'
#       },

s3.create_bucket(Bucket='mulani-athena',
                 CreateBucketConfiguration={
                    'LocationConstraint' : 'ap-south-1'
                 })

{'Location': 'http://mulani-athena.s3.amazonaws.com/',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sun, 03 May 2020 09:19:10 GMT',
   'location': 'http://mulani-athena.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'x-amz-id-2': 'olPBUPQBtiDW32IIIreyI4Fo9IaznVjEcS+qFk6go2CqLnLULgb8g04x12zofGCR+B8yRHN+oTg=',
   'x-amz-request-id': '0E85D5DF5839474C'},
  'HTTPStatusCode': 200,
  'HostId': 'olPBUPQBtiDW32IIIreyI4Fo9IaznVjEcS+qFk6go2CqLnLULgb8g04x12zofGCR+B8yRHN+oTg=',
  'RequestId': '0E85D5DF5839474C',
  'RetryAttempts': 0}}

In [5]:
athena = boto3.client('athena')

In [13]:
athena_result_config = {'OutputLocation': 's3://mulani-athena'}

In [17]:
# response = client.start_query_execution(
#       QueryString='string',
#       ClientRequestToken='string',
#       QueryExecutionContext={
#           'Database': 'string'
#       },
#       ResultConfiguration={
#           'OutputLocation': 'string',
#           'EncryptionConfiguration': {
#               'EncryptionOption': 'SSE_S3'|'SSE_KMS'|'CSE_KMS',
#               'KmsKey': 'string'
#           }
#       },
#       WorkGroup='string'
#   )

# CREATE DATABASE
athena.start_query_execution(
    QueryString='CREATE DATABASE mulani_athena_db',
    ResultConfiguration=athena_result_config
)

{'QueryExecutionId': '2b1d3611-50b0-4e22-a0a8-09cda1963adb',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '59',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 03 May 2020 09:35:13 GMT',
   'x-amzn-requestid': 'a91e5e4f-5d44-4f51-8515-a9bb0f673e3f'},
  'HTTPStatusCode': 200,
  'RequestId': 'a91e5e4f-5d44-4f51-8515-a9bb0f673e3f',
  'RetryAttempts': 0}}

In [25]:
# aws s3api put-object --bucket mulani-athena --key input/data.csv --body data.csv

# CREATE TABLE in the above database
schema_sql = '''
CREATE EXTERNAL TABLE IF NOT EXISTS mulani_athena_db.employee_table 
(name string, age int, salary int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION 's3://mulani-athena/input/'
TBLPROPERTIES ('has_encrypted_data'='false');
'''

athena.start_query_execution(
    QueryString=schema_sql,
    QueryExecutionContext={
        'Database': 'mulani_athena_db'
    },
    ResultConfiguration={
        'OutputLocation': 's3://mulani-athena'
    }
)

{'QueryExecutionId': '67a87a6f-99f4-4033-97b8-363376f0f4a2',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '59',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 03 May 2020 12:37:01 GMT',
   'x-amzn-requestid': 'b6969ef9-75f2-4d9c-ae10-bb6a3d4a8f5b'},
  'HTTPStatusCode': 200,
  'RequestId': 'b6969ef9-75f2-4d9c-ae10-bb6a3d4a8f5b',
  'RetryAttempts': 0}}